In [21]:
import os
import cv2
import numpy as np
from tqdm import tqdm

REBUILD_DATA = False

class Dogs_vs_Cats():
    IMG_SIZE = 50 
    CATS = "D:\\toydata\\PetImages\\Cat"
    DOGS = "D:\\toydata\\PetImages\\Dog"
    LABELS = {CATS: 0, DOGS: 1}
    
    training_data =[]
    catcount = 0
    dogcount = 0
    
    
    def make_training_data(self):
        for label in self.LABELS: 
            print(label)
            for f in tqdm(os.listdir(label)):

                try:
                    path = os.path.join(label, f)
                    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                    self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])
                    #one hot vector = np.eye(num_classes)[index of chosen class]

                    if label == self.CATS:
                        self.catcount += 1 

                    elif label == self.DOGS:
                        self.dogcount +=1
                        
                # some images are no good, skip them
                except Exception as e: 
                    pass
            
            np.random.shuffle(self.training_data)
            np.save("D:\\toydata\\PetImages\\training_data.npy", self.training_data)
            print("Cats:", self.catcount, "and Dogs:", self.dogcount)
            
if REBUILD_DATA:
    dogsvcats = Dogs_vs_Cats()
    dogsvcats.make_training_data()

training_data = np.load("D:\\toydata\\PetImages\\training_data.npy", allow_pickle=True)
print(len(training_data))

24946


In [13]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,32,5) # ..., features, kernal_size
        self.conv2 = nn.Conv2d(32,64,5) 
        self.conv3 = nn.Conv2d(64,128,5) 
         
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)
        
    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))

        print(x[0].shape)

        if self._to_linear is None: 
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
            # this is to get the input shape

        return x
        
    def forward (self, x):
        x = self.convs(x) # pass through all conv layers
        x = x.view(-1, self._to_linear_)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    
net = Net()

torch.Size([128, 2, 2])
